In [1]:
%load_ext autoreload
%autoreload 2
!sudo rm /tmp/libtpu_lockfile
import penzai
from penzai import pz
import jax.numpy as jnp
import jax_smi
import jax
jax_smi.initialise_tracking()
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

rm: cannot remove '/tmp/libtpu_lockfile': No such file or directory


In [2]:
from micrlhf.llama import LlamaTransformer
from micrlhf.scan import sequential_to_scan
from micrlhf.flash import flashify

import gc
llama = LlamaTransformer.from_pretrained(("models/llama-3-70b-1.gguf", "models/llama-3-70b-2.gguf"),
                                         device_map="auto:mp=4", load_on_cpu=True)
llama = sequential_to_scan(llama)
# llama = flashify(llama)

In [4]:
llama = llama.to_tpu()

In [5]:
import gc
gc.collect();

In [24]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("nvidia/Llama3-ChatQA-1.5-70B")
prompt = tokenizer.apply_chat_template([
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
] * 8, tokenize=False)

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
from penzai.toolshed import jit_wrapper
from micrlhf.flash import flashify
tokens = pz.nx.wrap([(tokenizer.encode(prompt))[:256]] * 4, "batch", "seq")
inputs = llama.inputs.from_basic_segments(tokens)
llama_jitted = jit_wrapper.Jitted(llama)

In [30]:
logits = llama_jitted(inputs)
loss = -pz.nx.nmap(lambda l, t: jnp.take_along_axis(jax.nn.log_softmax(l[:-1], -1), t[1:, None], 1).mean())(logits.untag("seq", "vocabulary"), tokens.untag("seq"))
loss

<NamedArray bfloat16(| batch:4) ≈1.3 ±0.0 [≥1.3, ≤1.3] nonzero:4 (wrapping jax.Array)>

In [9]:
from micrlhf.sampling import sample
sample(llama, tokenizer, prompt, batch_size=1, max_seq_len=128, do_sample=True,
       fold=True,
       )

ValueError: List arity mismatch: 9 != 0; list: [<NamedArray bfloat16(| layer:80, embedding:8192) ≈0.17 ±0.065 [≥-0.12, ≤0.74] nonzero:655_360 (wrapping jax.Array)>, <NamedArray float16(| layer:80, in_features:256, quant_group:1, mp:8192) ≈0.0 ±0.00049 [≥5.5e-06, ≤0.57] nonzero:167_772_160 (wrapping jax.Array)>, <NamedArray int8(| layer:80, in_features:256, quant_group:32, mp:8192) (wrapping jax.Array)>, <NamedArray float16(| layer:80, in_features:256, quant_group:1, mp:1024) ≈0.0 ±0.00091 [≥1.6e-05, ≤0.67] nonzero:20_971_520 (wrapping jax.Array)>, <NamedArray int8(| layer:80, in_features:256, quant_group:32, mp:1024) [≥-127, ≤127] zero:6_542_883 nonzero:664_545_757 (wrapping jax.Array)>, <NamedArray float16(| layer:80, in_features:256, quant_group:1, mp:1024) ≈0.0 ±0.002 [≥3.6e-05, ≤0.73] nonzero:20_971_520 (wrapping jax.Array)>, <NamedArray int8(| layer:80, in_features:256, quant_group:32, mp:1024) [≥-127, ≤127] zero:6_202_638 nonzero:664_886_002 (wrapping jax.Array)>, <NamedArray flo